In [1]:
from openai import OpenAI
from tqdm import tqdm
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:5035'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:5035'

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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"
)


In [2]:
def dict_to_string(d):
    """将字典转换为特定格式的字符串。

    参数:
    d (dict): 要转换的字典。

    返回:
    str: 转换后的字符串。
    """
    return '\n'.join(f"{i + 1}. {k}: {v}" for i, (k, v) in enumerate(d.items()))

# 示例使用
my_dict = {'apple': 'fruit', 'carrot': 'vegetable', 'chicken': 'meat'}
formatted_string = dict_to_string(my_dict)
print(formatted_string)

1. apple: fruit
2. carrot: vegetable
3. chicken: meat


In [3]:
def translate(text,vocab = {}, temperature=0.2, show = True):

    system_prompt = f"""
    - 你是一个世界级专业翻译家，你会准确、完整、流畅地翻译用户给你的文本。
- 你的专业领域是经济学和商业，你参与过WSJ和经济学人等专业财经杂志的翻译工作。
- 如果你做得好，我会给你$1000小费。
- 现在是5月，一年中生产力最高的时间。
- 用户会给你经济学的学术论文，你也采用经济学学术论文的风格进行翻译。
- 不要写“注”。
- 采用以下术语表:
{dict_to_string(vocab)}
- 严格忠实于原文，不要总结，只输出翻译结果。
    """.strip()

    example ="""去年,中国汽车海外销量激增,创下历史新高。"""

    stream = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "py4ss.net", # Optional, for including your app on openrouter.ai rankings.
            "X-Title": 'transltion', # Optional. Shows in rankings on openrouter.ai.
        },
        model="openai/gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "China’s overseas auto sales surged to a record last year."},
        {"role": "assistant", "content": example},
        {"role": "user", "content": text}],
        stream=True,
        temperature=temperature,
        frequency_penalty=0.8
    )

    result = ''

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            if show: print(chunk.choices[0].delta.content, end="")
            result += chunk.choices[0].delta.content

    return result

vocab = {'agent':'主体',
        'agent-based':'基于主体的',
        'multi-agent':'多主体',
        'actor':'主体',
        'mas':'多主体系统',
        'game': '博弈',
        'tick': '时间步'}


In [4]:
def rewriter_prompt(text1,text2):
    return f"""原文:
{text1}

译文:
{text2}

对上述中文译文进行重写，使其更为流畅，但保持准确性和学术风格。
直接输出重写后的中文内容，不要输出其他信息:"""

print(rewriter_prompt('adsf','中文'))


原文:
adsf

译文:
中文

对上述中文译文进行重写，使其更为流畅，但保持准确性和学术风格。
直接输出重写后的中文内容，不要输出其他信息:


In [5]:
def rewriter(text,vocab = {}, temperature=0.4, show = True):

    system_prompt = f"""
    - 你是一个世界级中英文期刊的编辑，你会对翻译后的文本进行重写。
- 你的专业领域是经济学和商业，你参与过WSJ和经济学人等专业财经杂志的编辑工作。
- 如果你做得好，我会给你$1000小费。
- 现在是5月，一年中生产力最高的时间。
- 用户会给你经济学的学术论文，你也采用经济学学术论文的风格。
- 采用以下术语表:
{dict_to_string(vocab)}
- 保留原文所有内容，只输出重写结果。
    """.strip()

    example_user_msg = """原文:
China’s overseas auto sales surged to a record last year.

译文:
中国海外汽车销售量去年激增至创纪录水平。

[参考原文，对上述中文译文进行重写:]"""

    example_output ="""去年,中国汽车海外销量激增,创下历史新高。"""

    stream = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "py4ss.net", # Optional, for including your app on openrouter.ai rankings.
            "X-Title": 'transltion', # Optional. Shows in rankings on openrouter.ai.
        },
        model="openai/gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": example_user_msg},
        {"role": "assistant", "content": example_output},
        {"role": "user", "content": text}],
        stream=True,
        temperature=temperature,
        frequency_penalty=0.5
    )

    result = ''

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            if show: print(chunk.choices[0].delta.content, end="")
            result += chunk.choices[0].delta.content

    return result


prompt = """原文:
In this paper, we aim to build a computational economic model that imitates the conditions of the French labor market in order to study the possible changes in the market, due to the introduction of a new type of a job contract. We intend to take judicious advantage of the tools provided by multi-agent systems (MAS), i.e. systems made of artificial agents that are autonomous and in strong interaction with each other,1 in order to translate an economic model to an agent-based one. The economic model is mainly described by differential equations, whereas in a multi-agent model each agent is an independent entity, constantly interacting with other agents and the environment. This transposition was elaborated in two phases. We started by identifying the main actors in the model. Each actor has his/her own main role, and his/her own behaviors (methods). These behaviors introduce the dynamics to the simulation. We have afterwards integrated the calculation mechanisms of the economic model, which take part in the decision processes of the agents.

译文:
本文旨在建立一个计算经济学模型，模拟法国劳动力市场的条件，以研究由于引入一种新的劳动合同而可能引起的市场变化。我们打算利用多主体系统（MAS）提供的工具，即由自治的人工主体组成，彼此之间有强烈互动的系统，将经济模型转换为基于主体的模型。经济模型主要由微分方程描述，而在多主体模型中，每个主体都是一个独立的实体，不断与其他主体和环境互动。这种转换分为两个阶段。我们首先确定了模型中的主要参与者。每个参与者都有自己的主要角色和自己的行为（方法）。这些行为为模拟引入了动态。之后，我们整合了经济模型的计算机制，这些机制参与主体的决策过程。

[参考原文，对上述中文译文进行重写:]"""

rewriter(prompt)


本文旨在构建一个计算经济模型，以模拟法国劳动力市场的条件，并研究引入一种新的工作合同可能导致的市场变化。我们打算充分利用多主体系统（MAS）提供的工具，即由自治且相互强烈交互的人工主体组成的系统，将经济模型转化为基于主体的模型。经济模型主要通过微分方程描述，而在多主体模型中，每个主体都是独立的实体，不断与其他主体和环境进行交互。这一转换分为两个阶段进行。首先，我们确定了模型中的主要参与者。每个参与者都有自己的主要角色和行为方式（方法）。这些行为方式为模拟引入了动态效应。然后，我们整合了经济模型中参与者决策过程中所涉及的计算机制。

'本文旨在构建一个计算经济模型，以模拟法国劳动力市场的条件，并研究引入一种新的工作合同可能导致的市场变化。我们打算充分利用多主体系统（MAS）提供的工具，即由自治且相互强烈交互的人工主体组成的系统，将经济模型转化为基于主体的模型。经济模型主要通过微分方程描述，而在多主体模型中，每个主体都是独立的实体，不断与其他主体和环境进行交互。这一转换分为两个阶段进行。首先，我们确定了模型中的主要参与者。每个参与者都有自己的主要角色和行为方式（方法）。这些行为方式为模拟引入了动态效应。然后，我们整合了经济模型中参与者决策过程中所涉及的计算机制。'

In [6]:
text = """In this paper, we aim to build a computational economic model that imitates the conditions of the French labor market in order to study the possible changes in the market, due to the introduction of a new type of a job contract. We intend to take judicious advantage of the tools provided by multi-agent systems (MAS), i.e. systems made of artificial agents that are autonomous and in strong interaction with each other,1 in order to translate an economic model to an agent-based one. The economic model is mainly described by differential equations, whereas in a multi-agent model each agent is an independent entity, constantly interacting with other agents and the environment. """

#text = """word"""
result = translate(text,
                   vocab = vocab,
                   temperature=0.7)
print('\n\n')
print(result)

本文旨在构建一个计算经济模型，以模拟法国劳动市场的条件，以研究由于引入新型的工作合同而可能发生的市场变化。我们打算充分利用多主体系统（MAS）提供的工具，即由人工智能代理组成、相互自治且强烈互动的系统，将经济模型转化为基于代理的模型。经济模型主要通过微分方程描述，而在多主体模型中，每个代理是一个独立实体，与其他代理和环境不断互动。


本文旨在构建一个计算经济模型，以模拟法国劳动市场的条件，以研究由于引入新型的工作合同而可能发生的市场变化。我们打算充分利用多主体系统（MAS）提供的工具，即由人工智能代理组成、相互自治且强烈互动的系统，将经济模型转化为基于代理的模型。经济模型主要通过微分方程描述，而在多主体模型中，每个代理是一个独立实体，与其他代理和环境不断互动。


In [7]:
def pre_process(text):
    return text.lstrip('# ').lstrip('#')

def after_process(text):
    text = text.replace('代理人','主体').replace('代理商','主体').replace('代理','主体').replace('智能体','主体').replace('<直译版>','').split('### Human')[0].strip()
    text = text.replace('演员','主体')
    return text


def translate_paper(input_file,output_file):

    with open(input_file,'r') as f:
        text = f.read()

    text = text.split('## References')[0]

    text_splitted = text.split('\n\n')

    text_splitted[:3]

    result = ''

    for sub_text in tqdm(text_splitted):
        # print('原文:' + sub_text)

        trans_text = ''

        if len(sub_text.strip()) == 0:
            continue
        if sub_text.strip()[0] != "#":

            sub_text_fix = pre_process(sub_text)

            text_to_trans = f"""前文和前文翻译:
    ...
    {result[-400:]}

    把下文翻译成中文，只输出译文:
    {sub_text_fix}"""

            #print("="*30,'\n',text_to_trans)

            trans_text = translate(text_to_trans, vocab = vocab, temperature=0.1,show=False)

            if len(trans_text) > 100:
                #print('[重写]')
                trans_text = rewriter(rewriter_prompt(sub_text_fix,trans_text),temperature=0.2,show=False)

            trans_text = after_process(trans_text)
        
        # print(trans_text)
        result += sub_text + "\n\n"
        
        result += trans_text + "\n\n"

        with open(output_file,'w') as f:
            f.write(result)

In [11]:
input_dir = 'platform_mds'
output_dir = 'platform_mds_tran'

md_files = [x for x in os.listdir(input_dir) if x[-2:]=='md']
md_files



['Sanchez-Cartas 和 León - 2021 - Multisided Platforms and Markets A Survey of the .md',
 'Jullien, Sand-Zantman - 2021 - The Economics of Platforms A Theory Guide for Competition Policy.md',
 'Sanchez-Cartas - 2018 - Agent-based models and industrial organization theory. A price-competition algorithm for agent-based models base.md',
 'Jullien 等 - 2021 - Two-sided markets, pricing, and network effects.md',
 'Rysman - 2009 - The economics of two-sided markets.md']

In [12]:

counter = 1
for md_file in md_files:
    print(str(counter) + '/' + str(len(md_files)))

    input_file = input_dir + '/' + md_file
    print(md_file)
    output_file = output_dir +'/' + md_file
    translate_paper(input_file, output_file)
    counter += 1


1/5
Sanchez-Cartas 和 León - 2021 - Multisided Platforms and Markets A Survey of the .md


100%|██████████| 137/137 [36:21<00:00, 15.92s/it]


2/5
Jullien, Sand-Zantman - 2021 - The Economics of Platforms A Theory Guide for Competition Policy.md


100%|██████████| 116/116 [29:36<00:00, 15.31s/it]


3/5
Sanchez-Cartas - 2018 - Agent-based models and industrial organization theory. A price-competition algorithm for agent-based models base.md


100%|██████████| 108/108 [20:54<00:00, 11.62s/it]


4/5
Jullien 等 - 2021 - Two-sided markets, pricing, and network effects.md


100%|██████████| 516/516 [1:41:41<00:00, 11.82s/it]  


5/5
Rysman - 2009 - The economics of two-sided markets.md


100%|██████████| 61/61 [16:16<00:00, 16.00s/it]
